# Update csv wind speed data using GFS

---

In [1]:
import os
import time
import glob

import getpass
import numpy as np
import rioxarray
import xarray as xr
import pandas as pd
import geopandas as gpd
import netCDF4 as nc

from maridatadownloader import DownloaderFactory

import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
import os
sys.path.append('../src/')
os.getcwd()

'/home/igor/projects/maridata/MariGeoRoute/data/maridatadownloader/notebooks'

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

# 1) GFS <a class="anchor" id="gfs"></a>
[go to TOC](#toc)

## PATHs data

In [3]:
POSITION_DATA = '../data/POSITION/'
WIND_SPEED_DATA = '../data/WIND_SPEED/'


## POSITION - join all csvs into one csv

In [4]:
def combine_position_csv_into_one(csv_file_list):
    '''return dataframe with data from all csvs in position directory'''
    
    # Create empty df 
    df_list = list()
    
    for csv_file in csv_file_list:
        # Read csv data 
        data = pd.read_csv(csv_file, names=['time','lat','N','lon','E'], sep='\||,')

        # Adapt coordinates from DDM to DD
        data['lat'] = data['lat'].map(lambda item: int(str(item)[:2]) + float(str(item)[2:])/60)
        data['lon'] = data['lon'].map(lambda item: int(str(item)[:2]) + float(str(item)[2:])/60)
        data.drop(columns=['N','E'], inplace=True)
        
        # Adding the df to df_list
        df_list.append(data)
       
    # Concatenate all dfs into one    
    result_df = pd.concat(df_list, axis=0)
    
    return result_df

In [5]:
position_list = glob.glob('../data/POSITION/2023*.csv')

combine_position_csv_into_one(position_list)

,time,lat,lon
0,2023-08-12T06:32:44.490,36.144592,15.044845
1,2023-08-12T06:32:45.799,36.144533,15.044805
2,2023-08-12T06:32:49.057,36.144245,15.044605
3,2023-08-12T06:32:50.097,36.144187,15.044565
4,2023-08-12T06:32:53.725,36.143957,15.044405
...,...,...,...
20995,2023-08-09T20:49:56.288,44.327818,12.925568
20996,2023-08-09T20:49:56.858,44.327818,12.925568
20997,2023-08-09T20:49:59.066,44.327742,12.925763
20998,2023-08-09T20:50:00.545,44.327717,12.925828


## WIND_DIRECTION - join all csvs into one csv

In [6]:
def combine_winddirection_csv_into_one(csv_file_list):
    
    # Create empty df 
    df_list = list()
    
    for csv_file in csv_file_list:
        # Read csv data 
        data = pd.read_csv(csv_file, names=['time', 'wind_dir'], sep='\||,')
        
        # Adding the df to df_list
        df_list.append(data)
       
    # Concatenate all dfs into one    
    result_df = pd.concat(df_list, axis=0)
    
    return result_df

In [7]:
wind_direction_list = glob.glob('../data/WIND_DIRECTION/2023*.csv')

combine_winddirection_csv_into_one(wind_direction_list)

,time,wind_dir
0,2023-08-11T21:01:25.187,249.3
1,2023-08-11T21:01:25.796,249.3
2,2023-08-11T21:01:26.936,249.3
3,2023-08-11T21:01:28.095,249.3
4,2023-08-11T21:01:29.744,249.3
...,...,...
33995,2023-08-09T03:08:54.115,249.3
33996,2023-08-09T03:08:56.864,249.3
33997,2023-08-09T03:08:59.053,249.3
33998,2023-08-09T03:09:00.612,249.3


## Common values between POSITION and WIND SPEED csvs

In [8]:
def verify_common_values(df_position, df_wind_direction):
    
    number = df_position['time'].isin(df_wind_direction['time']).value_counts()
    data = df_position[df_position['time'].isin(df_wind_direction['time'])]

    data.to_csv('common_values.csv')
    
    return number, data

In [9]:
verify_common_values(combine_position_csv_into_one(position_list), combine_winddirection_csv_into_one(wind_direction_list))[1]

,time,lat,lon
7203,2023-08-12T09:50:25.484,35.903060,14.778297
20029,2023-08-09T00:50:32.056,44.438960,12.244225
687,2023-08-11T02:03:10.095,40.607038,18.458875
4571,2023-08-13T03:47:49.316,35.921273,14.822353
2210,2023-08-09T02:18:43.674,44.438988,12.244212
9878,2023-08-09T05:52:26.537,44.438993,12.244180
13176,2023-08-09T07:22:24.122,44.438985,12.244208
2760,2023-08-10T07:40:42.758,42.999672,15.027075
15872,2023-08-10T13:43:30.903,42.243868,16.117480
3207,2023-08-11T22:20:53.174,37.399655,16.234945


## 1.1) Create downloader object <a class="anchor" id="gfs-init"></a>
[go to TOC](#toc)

In [10]:
gfs = DownloaderFactory.get_downloader('opendap', 'gfs')

In [11]:
gfs.dataset

<xarray.Dataset>
Dimensions:                                                                     (
                                                                                 lat: 721,
                                                                                 lon: 1440,
                                                                                 time: 116,
                                                                                 time1: 117,
                                                                                 time2: 218,
                                                                                 ...
                                                                                 depth_below_surface_layer_bounds_1: 2,
                                                                                 height_above_ground_layer_bounds_1: 2,
                                                                                 height_above_ground_layer1_bounds_1: 2,
                                                                                 pressure_difference_layer1_bounds_1: 2,
                                                                                 pressure_difference_layer2_bounds_1: 2,
                                                                                 sigma_layer_bounds_1: 2)
Coordinates: (12/41)
  * lat                                                                         (lat) float32 ...
  * lon                                                                         (lon) float32 ...
  * time                                                                        (time) datetime64[ns] ...
    reftime                                                                     (time) datetime64[ns] ...
  * time1                                                                       (time1) datetime64[ns] ...
    reftime1                                                                    (time1) datetime64[ns] ...
    ...                                                                          ...
    depth_below_surface_layer_bounds                                            (depth_below_surface_layer, depth_below_surface_layer_bounds_1) float32 ...
    height_above_ground_layer_bounds                                            (height_above_ground_layer, height_above_ground_layer_bounds_1) float32 ...
    height_above_ground_layer1_bounds                                           (height_above_ground_layer1, height_above_ground_layer1_bounds_1) float32 ...
    pressure_difference_layer1_bounds                                           (pressure_difference_layer1, pressure_difference_layer1_bounds_1) float32 ...
    pressure_difference_layer2_bounds                                           (pressure_difference_layer2, pressure_difference_layer2_bounds_1) float32 ...
    sigma_layer_bounds                                                          (sigma_layer, sigma_layer_bounds_1) float32 ...
Dimensions without coordinates: time2_bounds_1, time3_bounds_1,
                                pressure_difference_layer_bounds_1,
                                depth_below_surface_layer_bounds_1,
                                height_above_ground_layer_bounds_1,
                                height_above_ground_layer1_bounds_1,
                                pressure_difference_layer1_bounds_1,
                                pressure_difference_layer2_bounds_1,
                                sigma_layer_bounds_1
Data variables: (12/170)
    Total_ozone_entire_atmosphere_single_layer                                  (time1, lat, lon) float32 ...
    Ozone_Mixing_Ratio_isobaric                                                 (time1, isobaric, lat, lon) float32 ...
    Total_cloud_cover_entire_atmosphere_Mixed_intervals_Average                 (time3, lat, lon) float32 ...
    Low_cloud_cover_low_cloud_Mixed_intervals_Average                           (time3, lat, lon) float32 ...
 

## 1.2) Download data as datacube <a class="anchor" id="gfs-datacube"></a>
[go to TOC](#toc)

In [12]:
data_common_values = pd.read_csv('common_values.csv')
data_common_values

,Unnamed: 0,time,lat,lon
0,7203,2023-08-12T09:50:25.484,35.903060,14.778297
1,20029,2023-08-09T00:50:32.056,44.438960,12.244225
2,687,2023-08-11T02:03:10.095,40.607038,18.458875
3,4571,2023-08-13T03:47:49.316,35.921273,14.822353
4,2210,2023-08-09T02:18:43.674,44.438988,12.244212
5,9878,2023-08-09T05:52:26.537,44.438993,12.244180
6,13176,2023-08-09T07:22:24.122,44.438985,12.244208
7,2760,2023-08-10T07:40:42.758,42.999672,15.027075
8,15872,2023-08-10T13:43:30.903,42.243868,16.117480
9,3207,2023-08-11T22:20:53.174,37.399655,16.234945


In [13]:
def find_center(x, y):
    coordinates_center = []    
    xCenter = np.sum(x)/len(y)
    yCenter = np.sum(y)/len(y)
    coordinates_center.append(xCenter)
    coordinates_center.append(yCenter)

    np.array(coordinates_center)
    return coordinates_center

In [16]:
# Configuration for downloading as cube
time_min = data_common_values['time'].min()
time_max = data_common_values['time'].max()

#height_min = 10
#height_max = 10
lon_min=data_common_values['lon'].min().round()
lon_max=data_common_values['lon'].max().round()+1
lat_min=data_common_values['lat'].min().round()
lat_max=data_common_values['lat'].max().round()+1


# Convert to datetime to round seconds and convert to string again
data_common_values['time'] = pd.to_datetime(data_common_values['time']).round('1s').astype('string')  #match format '%Y-%m-%d %H:%M:%S'
    
# Select parameters 
parameters = ["u-component_of_wind_height_above_ground", "v-component_of_wind_height_above_ground"]

sel_dict = {'time': slice(time_min, time_max), 'time1': slice(time_min, time_max),
           'latitude': slice(lat_min, lat_max), 'longitude': slice(lon_min, lon_max)}



In [17]:
dataset_gfs = gfs.download(parameters=parameters, sel_dict=sel_dict)
dataset_gfs

Access archived GFS data


<xarray.Dataset>
Dimensions:                                  (time: 33, height_above_ground: 7,
                                              latitude: 37, longitude: 29)
Coordinates:
  * latitude                                 (latitude) float32 36.0 ... 45.0
  * longitude                                (longitude) float32 12.0 ... 19.0
  * time                                     (time) datetime64[ns] 2023-08-09...
  * height_above_ground                      (height_above_ground) float32 10...
    LatLon_Projection                        int32 0
    reftime                                  (time) datetime64[ns] 2023-08-09...
Data variables:
    u-component_of_wind_height_above_ground  (time, height_above_ground, latitude, longitude) float32 ...
    v-component_of_wind_height_above_ground  (time, height_above_ground, latitude, longitude) float32 ...
Attributes:
    Originating_or_generating_Center:                                        ...
    Originating_or_generating_Subcenter:                                     ...
    GRIB_table_version:                                                      ...
    Type_of_generating_process:                                              ...
    Analysis_or_forecast_generating_process_identifier_defined_by_originating...
    file_format:                                                             ...
    Conventions:                                                             ...
    history:                                                                 ...
    featureType:                                                             ...
    EXTRA_DIMENSION.reftime:                                                 ...

In [135]:
# Select only 10m height above ground
dataset_gfs = dataset_gfs.sel(height_above_ground = 10)
dataset_gfs

<xarray.Dataset>
Dimensions:                                  (time: 33, latitude: 37,
                                              longitude: 29)
Coordinates:
  * latitude                                 (latitude) float32 36.0 ... 45.0
  * longitude                                (longitude) float32 12.0 ... 19.0
  * time                                     (time) datetime64[ns] 2023-08-09...
    height_above_ground                      float32 10.0
    LatLon_Projection                        int32 0
    reftime                                  (time) datetime64[ns] 2023-08-09...
Data variables:
    u-component_of_wind_height_above_ground  (time, latitude, longitude) float32 ...
    v-component_of_wind_height_above_ground  (time, latitude, longitude) float32 ...
Attributes:
    Originating_or_generating_Center:                                        ...
    Originating_or_generating_Subcenter:                                     ...
    GRIB_table_version:                                                      ...
    Type_of_generating_process:                                              ...
    Analysis_or_forecast_generating_process_identifier_defined_by_originating...
    file_format:                                                             ...
    Conventions:                                                             ...
    history:                                                                 ...
    featureType:                                                             ...
    EXTRA_DIMENSION.reftime:                                                 ...

In [18]:
# Remove unnecessary coordinates
if 'reftime' in dataset_gfs.coords:
    dataset_gfs = dataset_gfs.drop('reftime')
if 'LatLon_Projection' in dataset_gfs.coords:   
    dataset_gfs = dataset_gfs.drop('LatLon_Projection')

In [19]:
dataset_gfs

<xarray.Dataset>
Dimensions:                                  (time: 33, height_above_ground: 7,
                                              latitude: 37, longitude: 29)
Coordinates:
  * latitude                                 (latitude) float32 36.0 ... 45.0
  * longitude                                (longitude) float32 12.0 ... 19.0
  * time                                     (time) datetime64[ns] 2023-08-09...
  * height_above_ground                      (height_above_ground) float32 10...
Data variables:
    u-component_of_wind_height_above_ground  (time, height_above_ground, latitude, longitude) float32 ...
    v-component_of_wind_height_above_ground  (time, height_above_ground, latitude, longitude) float32 ...
Attributes:
    Originating_or_generating_Center:                                        ...
    Originating_or_generating_Subcenter:                                     ...
    GRIB_table_version:                                                      ...
    Type_of_generating_process:                                              ...
    Analysis_or_forecast_generating_process_identifier_defined_by_originating...
    file_format:                                                             ...
    Conventions:                                                             ...
    history:                                                                 ...
    featureType:                                                             ...
    EXTRA_DIMENSION.reftime:                                                 ...

In [23]:
# Save dataset to .nc file 
if os.path.isfile('gfs_bbox.nc'):
    os.remove('gfs_bbox.nc')
    dataset_gfs.to_netcdf('gfs_bbox.nc')
else:
    dataset_gfs.to_netcdf('gfs_bbox.nc')

In [31]:
nc_path = "./gfs_bbox.nc"
csv_path = "./common_values.csv"
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)
# Create a GeoDataFrame from the DataFrame with lat/lon as geometry
geometry = gpd.points_from_xy(df['lon'], df['lat'])
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326').iloc[:, 1:]  # Assuming WGS84 coordinates
gdf

,time,lat,lon,geometry
0,2023-08-12T09:50:25.484,35.903060,14.778297,POINT (14.77830 35.90306)
1,2023-08-09T00:50:32.056,44.438960,12.244225,POINT (12.24423 44.43896)
2,2023-08-11T02:03:10.095,40.607038,18.458875,POINT (18.45887 40.60704)
3,2023-08-13T03:47:49.316,35.921273,14.822353,POINT (14.82235 35.92127)
4,2023-08-09T02:18:43.674,44.438988,12.244212,POINT (12.24421 44.43899)
5,2023-08-09T05:52:26.537,44.438993,12.244180,POINT (12.24418 44.43899)
6,2023-08-09T07:22:24.122,44.438985,12.244208,POINT (12.24421 44.43899)
7,2023-08-10T07:40:42.758,42.999672,15.027075,POINT (15.02707 42.99967)
8,2023-08-10T13:43:30.903,42.243868,16.117480,POINT (16.11748 42.24387)
9,2023-08-11T22:20:53.174,37.399655,16.234945,POINT (16.23494 37.39966)


In [39]:
# Try to sample points from a raster
# Define paths to your NetCDF file and CSV file
nc_path = "./gfs_bbox.nc"
csv_path = "./common_values.csv"
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)
# Create a GeoDataFrame from the DataFrame with lat/lon as geometry
geometry = gpd.points_from_xy(df['lon'], df['lat'])
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')  # Assuming WGS84 coordinates
# Open the NetCDF file using netCDF4
with nc.Dataset(nc_path, 'r') as dataset:
    for index, row in gdf.iterrows():
        lon, lat = row['lon'], row['lat']
        point = gdf.geometry.iloc[index]
        # Extract the raster values at the point's coordinates
        lon_index = (abs(dataset.variables['lon'][:] - lon)).argmin()
        lat_index = (abs(dataset.variables['lat'][:] - lat)).argmin()
        raster_value = dataset.variables['variable_name'][0, lat_index, lon_index]
        # Add the raster value to the GeoDataFrame
        gdf.at[index, 'RasterValue'] = raster_value
# Save the GeoDataFrame to a shapefile or another desired format
output_path = "result_combination_csv_nc.shp"
gdf.to_file(output_path)
print("Raster values extracted from the NetCDF file and saved to the output shapefile.")

KeyError: 'lon'

In [34]:
geometry = gpd.points_from_xy(data_common_values['lon'], data_common_values['lat'])

gdf = gpd.GeoDataFrame(data_common_values, geometry=geometry, crs='EPSG:4326').iloc[:, 1:]
gdf

,time,lat,lon,geometry
0,2023-08-12 09:50:25,35.903060,14.778297,POINT (14.77830 35.90306)
1,2023-08-09 00:50:32,44.438960,12.244225,POINT (12.24423 44.43896)
2,2023-08-11 02:03:10,40.607038,18.458875,POINT (18.45887 40.60704)
3,2023-08-13 03:47:49,35.921273,14.822353,POINT (14.82235 35.92127)
4,2023-08-09 02:18:44,44.438988,12.244212,POINT (12.24421 44.43899)
5,2023-08-09 05:52:27,44.438993,12.244180,POINT (12.24418 44.43899)
6,2023-08-09 07:22:24,44.438985,12.244208,POINT (12.24421 44.43899)
7,2023-08-10 07:40:43,42.999672,15.027075,POINT (15.02707 42.99967)
8,2023-08-10 13:43:31,42.243868,16.117480,POINT (16.11748 42.24387)
9,2023-08-11 22:20:53,37.399655,16.234945,POINT (16.23494 37.39966)


Since trying to extract the raster value for a specific coordinate was not working, this step was done with the "Point sampling tool" in QGIS for this data. The workflow to do this was:
- Add vector data (common_values.csv file created by using the function verify_common_values())
- Add raster data (gfs_bbox.nc)
- Apply the "point sampling tool" to all the layers of u and v direction
- Generate the csv_nc_points.gpkg

In [137]:
gdf_result = gpd.read_file('../data/csv_nc_points.gpkg')
gdf_result

,time,lat,lon,gfs_bbox — u-component_of_wind_height_above_ground_1,gfs_bbox — u-component_of_wind_height_above_ground_2,gfs_bbox — u-component_of_wind_height_above_ground_3,gfs_bbox — u-component_of_wind_height_above_ground_4,gfs_bbox — u-component_of_wind_height_above_ground_5,gfs_bbox — u-component_of_wind_height_above_ground_6,gfs_bbox — u-component_of_wind_height_above_ground_7,...,gfs_bbox — v-component_of_wind_height_above_ground_25,gfs_bbox — v-component_of_wind_height_above_ground_26,gfs_bbox — v-component_of_wind_height_above_ground_27,gfs_bbox — v-component_of_wind_height_above_ground_28,gfs_bbox — v-component_of_wind_height_above_ground_29,gfs_bbox — v-component_of_wind_height_above_ground_30,gfs_bbox — v-component_of_wind_height_above_ground_31,gfs_bbox — v-component_of_wind_height_above_ground_32,gfs_bbox — v-component_of_wind_height_above_ground_33,geometry
0,2023-08-12 09:50:25,35.903060,14.778297,1.17782,2.08992,1.73676,2.4436,3.96276,4.70745,4.04609,...,-1.53331,-0.95474,-0.85902,-0.2578,0.99979,0.26912,-0.2251,-0.87571,-1.47371,POINT (14.77830 35.90306)
1,2023-08-09 00:50:32,44.438960,12.244225,0.12782,2.52992,1.04676,-2.1264,-4.22724,-3.59255,-3.14391,...,1.52669,1.25526,0.17098,0.2722,1.55979,2.41912,1.8749,1.82429,1.02629,POINT (12.24423 44.43896)
2,2023-08-11 02:03:10,40.607038,18.458875,0.89782,-0.32008,0.55676,1.3136,2.07276,1.09745,0.67609,...,-4.58331,-4.39474,-3.74902,-5.1878,-7.51021,-5.94088,-6.1951,-6.63571,-7.08371,POINT (18.45887 40.60704)
3,2023-08-13 03:47:49,35.921273,14.822353,1.17782,2.08992,1.73676,2.4436,3.96276,4.70745,4.04609,...,-1.53331,-0.95474,-0.85902,-0.2578,0.99979,0.26912,-0.2251,-0.87571,-1.47371,POINT (14.82235 35.92127)
4,2023-08-09 02:18:43,44.438988,12.244212,0.12782,2.52992,1.04676,-2.1264,-4.22724,-3.59255,-3.14391,...,1.52669,1.25526,0.17098,0.2722,1.55979,2.41912,1.8749,1.82429,1.02629,POINT (12.24421 44.43899)
5,2023-08-09 05:52:26,44.438993,12.244180,0.12782,2.52992,1.04676,-2.1264,-4.22724,-3.59255,-3.14391,...,1.52669,1.25526,0.17098,0.2722,1.55979,2.41912,1.8749,1.82429,1.02629,POINT (12.24418 44.43899)
6,2023-08-09 07:22:24,44.438985,12.244208,0.12782,2.52992,1.04676,-2.1264,-4.22724,-3.59255,-3.14391,...,1.52669,1.25526,0.17098,0.2722,1.55979,2.41912,1.8749,1.82429,1.02629,POINT (12.24421 44.43899)
7,2023-08-10 07:40:42,42.999672,15.027075,0.81782,-1.63008,-0.33324,0.2936,-0.59724,-0.63255,1.82609,...,-2.31331,-3.79474,-4.11902,-4.0678,-2.78021,-2.22088,-1.5551,-2.32571,-1.31371,POINT (15.02707 42.99967)
8,2023-08-10 13:43:30,42.243868,16.117480,2.11782,1.20992,0.60676,0.8336,-1.36724,-0.86255,2.48609,...,-1.33331,-0.20474,-1.82902,-1.5978,-0.78021,-2.85088,-2.6351,-1.41571,-1.35371,POINT (16.11748 42.24387)
9,2023-08-11 22:20:53,37.399655,16.234945,0.68782,0.15992,0.86676,0.8536,2.93276,3.85745,5.30609,...,-3.37331,-2.33474,-2.52902,0.3122,-2.04021,-3.15088,-3.8651,-4.23571,-6.61371,POINT (16.23494 37.39966)


In [138]:
# Calculate u mean value
gdf_result['u_avg'] = gdf_result[gdf_result.columns[3:36]].mean(axis=1)

# Calculate v mean value
gdf_result['v_avg'] = gdf_result[gdf_result.columns[36:-2]].mean(axis=1)

# Calculate wind speed magnitude ws = sqrt(u2+v2) 
gdf_result['ws'] = np.sqrt(gdf_result['u_avg']**2 + gdf_result['v_avg']**2)

# Calculate wind direction (REVIEW)
gdf_result['wd'] = np.arctan2(gdf_result['v_avg'], gdf_result['u_avg'])

# Drop unnecessary columns
gdf_result.drop(columns=gdf_result.iloc[:, 3:-4],inplace=True)

gdf_result

,time,lat,lon,u_avg,v_avg,ws,wd
0,2023-08-12 09:50:25,35.903060,14.778297,0.712909,-0.421321,0.828101,-0.533767
1,2023-08-09 00:50:32,44.438960,12.244225,-2.698000,0.710800,2.790061,2.883992
2,2023-08-11 02:03:10,40.607038,18.458875,1.372303,-6.341321,6.488110,-1.357676
3,2023-08-13 03:47:49,35.921273,14.822353,0.712909,-0.421321,0.828101,-0.533767
4,2023-08-09 02:18:43,44.438988,12.244212,-2.698000,0.710800,2.790061,2.883992
5,2023-08-09 05:52:26,44.438993,12.244180,-2.698000,0.710800,2.790061,2.883992
6,2023-08-09 07:22:24,44.438985,12.244208,-2.698000,0.710800,2.790061,2.883992
7,2023-08-10 07:40:42,42.999672,15.027075,0.507758,-2.818594,2.863964,-1.392562
8,2023-08-10 13:43:30,42.243868,16.117480,1.718061,-2.524351,3.053536,-0.973213
9,2023-08-11 22:20:53,37.399655,16.234945,-0.062545,-1.089503,1.091297,-1.628140


In [139]:
gdf_result.to_csv('result_wind_speed_uv.csv')

In [140]:
# Read all wind speed csv f
wind_data = combine_winddirection_csv_into_one(wind_direction_list)

# Convert to datetime to round seconds and convert to string again
wind_data['time'] = pd.to_datetime(wind_data['time']).round('1s')

wind_data

,time,wind_dir
0,2023-08-11 21:01:25,249.3
1,2023-08-11 21:01:26,249.3
2,2023-08-11 21:01:27,249.3
3,2023-08-11 21:01:28,249.3
4,2023-08-11 21:01:30,249.3
...,...,...
33995,2023-08-09 03:08:54,249.3
33996,2023-08-09 03:08:57,249.3
33997,2023-08-09 03:08:59,249.3
33998,2023-08-09 03:09:01,249.3


In [141]:
# Merge initial wind csv data and results gdf
merged = pd.merge(wind_data, gdf_result, on='time', how='left')
merged = merged.loc[(merged['lat'].notna() == True)]
merged = merged.reset_index().drop(columns=['index'])
merged

,time,wind_dir,lat,lon,u_avg,v_avg,ws,wd
0,2023-08-11 22:20:53,249.3,37.399655,16.234945,-0.062545,-1.089503,1.091297,-1.628140
1,2023-08-12 02:52:30,249.3,36.721667,15.585500,0.557758,-0.168594,0.582681,-0.293538
2,2023-08-12 09:50:25,249.3,35.903060,14.778297,0.712909,-0.421321,0.828101,-0.533767
3,2023-08-09 07:22:24,249.3,44.438985,12.244208,-2.698000,0.710800,2.790061,2.883992
4,2023-08-09 17:02:42,249.3,44.438978,12.244160,-2.698000,0.710800,2.790061,2.883992
5,2023-08-10 13:43:30,249.3,42.243868,16.117480,1.718061,-2.524351,3.053536,-0.973213
6,2023-08-11 01:31:05,249.3,40.706087,18.407235,1.060182,-6.004048,6.096932,-1.396020
7,2023-08-11 02:03:10,249.3,40.607038,18.458875,1.372303,-6.341321,6.488110,-1.357676
8,2023-08-11 19:10:27,249.3,37.855698,16.678265,-0.206485,-2.360715,2.369728,-1.658041
9,2023-08-09 23:50:40,249.3,44.006510,13.588097,-1.206787,-0.570412,1.334805,-2.700047


In [397]:
merged.to_csv('merged.csv')

In [157]:
# Apply values to all rows in wind_speed dataframe considering nearest time 
wind_speed_new = pd.merge_asof(wind_data.sort_values('time'), merged.sort_values('time'), on='time', direction='nearest')
wind_speed_new.drop(columns=['wind_dir_x', 'wind_dir_y'], inplace=True)
wind_speed_new

,time,lat,lon,u_avg,v_avg,ws,wd
0,2023-08-08 10:52:34,44.438960,12.244225,-2.698000,0.710800,2.790061,2.883992
1,2023-08-08 10:52:36,44.438960,12.244225,-2.698000,0.710800,2.790061,2.883992
2,2023-08-08 10:52:37,44.438960,12.244225,-2.698000,0.710800,2.790061,2.883992
3,2023-08-08 10:52:38,44.438960,12.244225,-2.698000,0.710800,2.790061,2.883992
4,2023-08-08 10:52:39,44.438960,12.244225,-2.698000,0.710800,2.790061,2.883992
...,...,...,...,...,...,...,...
271995,2023-08-13 22:25:54,35.921273,14.822353,0.712909,-0.421321,0.828101,-0.533767
271996,2023-08-13 22:25:54,35.921273,14.822353,0.712909,-0.421321,0.828101,-0.533767
271997,2023-08-13 22:25:58,35.921273,14.822353,0.712909,-0.421321,0.828101,-0.533767
271998,2023-08-13 22:26:00,35.921273,14.822353,0.712909,-0.421321,0.828101,-0.533767


In [158]:
# Save to csv file 
wind_speed_new.to_csv('wind_speed_new.csv')